### Import Dependencies

In [2]:
import quandl

import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import style
from matplotlib import pyplot as plt
from scipy import optimize as opt

pd.options.display.max_rows = 10
pd.options.display.float_format = "{2:.f}".format
style.use('fivethirtyeight')
np.random.seed(42)

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [3]:
with open('../../auth.txt') as my_api:
    key = my_api.read()

quandl.ApiConfig.api_key = key